In [ ]:
!pip install glove_python

In [ ]:
from glove import Corpus, Glove

In [ ]:
### read our custom data 

In [13]:
corpus = pd.read_csv('processed_Dtrain_4f.csv')

In [14]:
corpus['title'] = [ '' if x is np.NaN else x for x in corpus['title'] ]
corpus['match_title'] = [ '' if x is np.NaN else x for x in corpus['match_title'] ]
corpus['description'] = [ '' if x is np.NaN else x for x in corpus['description'] ]
corpus['match_description'] = [ '' if x is np.NaN else x for x in corpus['match_description'] ]
corpus['color'] = [ '' if x is np.NaN else x for x in corpus['color'] ]

In [15]:
import nltk
from nltk.tokenize import word_tokenize

corpus['title'] = corpus.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)
corpus['match_title'] = corpus.apply(lambda row: nltk.word_tokenize(row['match_title']), axis=1)
corpus['description'] = corpus.apply(lambda row: nltk.word_tokenize(row['description']), axis=1)
corpus['match_description'] = corpus.apply(lambda row: nltk.word_tokenize(row['match_description']), axis=1)
corpus['color'] = corpus.apply(lambda row: nltk.word_tokenize(row['color']), axis=1)

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [16]:
sentences = []
for row in corpus['title']:
    sentences.append(list(row))
    
for row in corpus['match_title']:
    sentences.append(list(row))

for row in corpus['description']:
    sentences.append(list(row))
    
for row in corpus['match_description']:
    sentences.append(list(row))
    
for row in corpus['color']:
    sentences.append(list(row))

### Training Glove on custom data + using pre-trained model as base

In [5]:
from gensim.scripts.glove2word2vec import glove2word2vec
import numpy as np
from gensim.test.utils import get_tmpfile

In [10]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [ ]:
root_folder='.'
data_folder_name='data'
glove_filename='glove.6B.100d.txt'
word2vec_output_file = glove_filename+'.word2vec'

In [3]:
glove_file = './glove.6B.100d.txt'

In [8]:
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [18]:
# build a word2vec model on your dataset 

base_model = Word2Vec(size=200, min_count=1)
base_model.build_vocab(sentences)  ## our custom dataset
total_examples = base_model.corpus_count

In [20]:
# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on your data
base_model.train(sentences, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv

In [26]:
len(base_model_wv.vocab)

451771

In [33]:
base_model_wv.most_similar('shoe')

[('sneaker', 0.6576837301254272),
 ('shoes', 0.6188355684280396),
 ('loafer', 0.6098387241363525),
 ('sandal', 0.589873731136322),
 ('mule', 0.5402063131332397),
 ('bootie', 0.5333376526832581),
 ('derby', 0.5296701192855835),
 ('boot', 0.5198146104812622),
 ('sneakers', 0.4869774281978607),
 ('wingtip', 0.4727979004383087)]

### Bi-gram / Tri-gram model 

In [ ]:
# bi-grams
bigram = Phrases(sentences, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

In [ ]:
## Tri-grams
trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

In [ ]:
# build a toy model to update with
model = Word2Vec(size=200, min_count=1)
model.build_vocab(trigramed_tokens)
total_examples = model.corpus_count

# add GloVe's vocabulary & weights
model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
model_wv = model.wv